In [27]:
import pandas as pd
import plotly.graph_objects as go

# Load data
game_week_scores = pd.read_csv('../CSV/10.02.2026/game_week_scores_rows.csv')
game_weeks = pd.read_csv('../CSV/10.02.2026/game_weeks_rows.csv')
profiles = pd.read_csv('../CSV/19.01.2026/profiles.csv')

# Merge data
merged = game_week_scores.merge(
    game_weeks[['id', 'week_number']], 
    left_on='game_week_id', 
    right_on='id', 
    how='left'
).merge(
    profiles[['id', 'username']], 
    left_on='player_id', 
    right_on='id', 
    how='left',
    suffixes=('', '_profile')
)

# Keep relevant columns
df = merged[['username', 'week_number', 'points', 'correct_scores']].copy()

print(f"Data shape: {df.shape}")
print(f"Weeks: {sorted(df['week_number'].unique())}")
print(f"Players: {len(df['username'].unique())}")
df.head()

Data shape: (783, 4)
Weeks: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23)]
Players: 35


,username,week_number,points,correct_scores
0,Matt Lavery,23,1,0
1,RogerStanton,11,7,0
2,The General,9,5,0
3,Sid Elliott,2,9,2
4,Des McCarthy,16,4,0


In [28]:
# Calculate cumulative points and correct scores for each player per week
# Remove test player
df = df[df['username'] != 'Martinez']

df_sorted = df.sort_values(['username', 'week_number'])

df_sorted['cumulative_points'] = df_sorted.groupby('username')['points'].cumsum()
df_sorted['cumulative_correct_scores'] = df_sorted.groupby('username')['correct_scores'].cumsum()

# For each week, calculate league position
def calculate_positions(week_df):
    # Sort by cumulative points (desc), cumulative correct scores (desc), then username (asc)
    week_df = week_df.sort_values(
        ['cumulative_points', 'cumulative_correct_scores', 'username'], 
        ascending=[False, False, True]
    )
    week_df['position'] = range(1, len(week_df) + 1)
    return week_df

# Apply to each week
league_positions = df_sorted.groupby('week_number', group_keys=False).apply(calculate_positions)

# Pivot to get position by week for each player
position_by_week = league_positions.pivot(index='username', columns='week_number', values='position')

print("League positions by week:")
print(position_by_week.head())
position_by_week

League positions by week:
week_number       1   2   3   4   5   6   7   8   9   10  ...  14  15  16  17  \
username                                                  ...                   
Alan Taylor-Reed  15  17  18  21  24  27  30  26  29  30  ...  31  31  31  29   
Andy Page         21  21  26  30  33  34  32  33  32  32  ...  33  33  33  32   
Bob sullivan      16   5   5  10   7   7  12  16  21  27  ...  29  29  23  26   
Bryan             10  15  10  11  10  16  20  20  22  18  ...  17  17  25  24   
CFC Stew          28  31  23  18   3   4   5   6   5   5  ...  11  11   7   5   

week_number       18  19  20  21  22  23  
username                                  
Alan Taylor-Reed  29  27  28  29  30  30  
Andy Page         32  31  32  31  32  31  
Bob sullivan      21  12  12  14  15  12  
Bryan             26  24  25  23  21  15  
CFC Stew           6   9   7   7  10  11  

[5 rows x 23 columns]


C:\Users\gavin\AppData\Local\Temp\ipykernel_20016\1369890602.py:21: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



week_number,1,2,3,4,5,6,7,8,9,10,...,14,15,16,17,18,19,20,21,22,23
username,,,,,,,,,,,,,,,,,,,,,
Alan Taylor-Reed,15,17,18,21,24,27,30,26,29,30,...,31,31,31,29,29,27,28,29,30,30
Andy Page,21,21,26,30,33,34,32,33,32,32,...,33,33,33,32,32,31,32,31,32,31
Bob sullivan,16,5,5,10,7,7,12,16,21,27,...,29,29,23,26,21,12,12,14,15,12
Bryan,10,15,10,11,10,16,20,20,22,18,...,17,17,25,24,26,24,25,23,21,15
CFC Stew,28,31,23,18,3,4,5,6,5,5,...,11,11,7,5,6,9,7,7,10,11
Chris Torode,33,24,25,17,12,10,4,8,8,3,...,5,5,8,11,7,10,10,10,6,7
Des McCarthy,6,18,19,27,31,31,34,34,33,33,...,32,32,32,33,33,33,33,33,31,32
Gerard,1,1,2,2,1,2,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Graham Dongworth,24,27,20,19,25,18,22,11,18,22,...,13,14,11,16,19,20,15,17,20,22


In [29]:
# Create interactive line chart with Plotly
fig = go.Figure()

# Define bright neon colors for lines
neon_colors = [
    '#FF00FF', '#00FFFF', '#00FF00', '#FFFF00', '#FF0080', 
    '#80FF00', '#0080FF', '#FF8000', '#FF0040', '#40FF00',
    '#00FF80', '#8000FF', '#FF0000', '#00FF40', '#4000FF',
    '#FF4000', '#00FFFF', '#FF00BF', '#BFFF00', '#00BFFF',
    '#FF00FF', '#00FF00', '#FFBF00', '#BF00FF', '#00FFBF',
    '#FF80FF', '#80FFFF', '#FFFF80', '#FF80C0', '#C0FF80',
    '#80C0FF', '#FFC080', '#C080FF', '#80FFC0'
]

# Get sorted player names (alphabetically)
players = sorted(position_by_week.index)

# Add a line for each player (start with all hidden)
for idx, player in enumerate(players):
    player_positions = position_by_week.loc[player]
    weeks = player_positions.index.tolist()
    positions = player_positions.values.tolist()
    
    fig.add_trace(go.Scatter(
        x=weeks,
        y=positions,
        mode='lines+markers',
        name=player,
        line=dict(width=2.5, color=neon_colors[idx % len(neon_colors)]),
        marker=dict(size=7, color=neon_colors[idx % len(neon_colors)]),
        visible='legendonly'
    ))

# Update layout with dark theme
fig.update_layout(
    title=dict(
        text='League Position Progression Throughout the Season',
        font=dict(color='#E0E0E0', size=18)
    ),
    xaxis_title='Game Week',
    yaxis_title='League Position',
    yaxis=dict(
        autorange='reversed',
        range=[0, 35],
        dtick=5,
        gridcolor='#404040',
        color='#E0E0E0',
        zeroline=False,
        showline=False,
        mirror=False  # Don't mirror axis on opposite side
    ),
    xaxis=dict(
        dtick=1,
        gridcolor='#404040',
        color='#E0E0E0',
        range=[0, len(position_by_week.columns) + 1],
        showline=False  # Hide the x-axis line
    ),
    plot_bgcolor='#1A1A1A',
    paper_bgcolor='#000000',
    font=dict(color='#C0C0C0'),
    hovermode='closest',
    legend=dict(
        title=dict(text='Players (click to toggle)', font=dict(color='#E0E0E0')),
        yanchor='top',
        y=1,
        xanchor='left',
        x=1.02,
        bgcolor='rgba(0,0,0,0.7)',
        bordercolor='#404040',
        borderwidth=1,
        font=dict(color='#C0C0C0')
    ),
    height=600,
    showlegend=True,
    updatemenus=[
        dict(
            type='buttons',
            direction='left',
            buttons=[
                dict(
                    label='Show All',
                    method='update',
                    args=[{'visible': True}]
                ),
                dict(
                    label='Hide All',
                    method='update',
                    args=[{'visible': 'legendonly'}]
                )
            ],
            x=1.0,
            xanchor='right',
            y=1.15,
            yanchor='top',
            bgcolor='#2A2A2A',
            bordercolor='#404040',
            font=dict(color='#000000')  # Black text for better contrast on white active state
        )
    ]
)

fig.show()

## Export Chart

In [30]:
from datetime import datetime

# Generate filename with today's date (DDMMYY format)
date_str = datetime.now().strftime('%d%m%y')
filename = f'LeagueProgression{date_str}.html'
filepath = f'../Output/{filename}'

# Export to Output directory
fig.write_html(filepath)
print(f"✓ Chart exported to: {filepath}")

✓ Chart exported to: ../Output/LeagueProgression120226.html
